In [34]:
using JuMP
using Ipopt
using GLPK
using GamsStructure

This problem was passed to my by my colleague Phil Graves.  I also need to 
thank Erwin Kalvalagen for teaching me what is meant by a "cut" in integer
programming.  

Thomas Rutherford
Economics Department
University of Colorado

GIVEN

1. In a street there are five houses, painted five different colours. 
2. In each house lives a person of different nationality 
3. These five homeowners each drink a different kind of beverage, smoke 
different brand of cigar and keep a different pet. 

THE QUESTION: WHO OWNS THE FISH? 

HINTS 

1. The Brit lives in a red house. 
2. The Swede keeps dogs as pets. 
3. The Dane drinks tea. 
4. The Green house is just to the left of the White house. 
5. The owner of the Green house drinks coffee. 
6. The person who smokes Pall Mall rears birds. 
7. The owner of the Yellow house smokes Dunhill. 
8. The man living in the centre house drinks milk. 
9. The Norwegian lives in the first house. 
10. The man who smokes Blends lives next to the one who keeps cats. 
11. The man who keeps horses lives next to the man who smokes Dunhill. 
12. The man who smokes Blue Master drinks beer. 
13. The German smokes Prince. 
14. The Norwegian lives next to the blue house. 
15. The man who smokes Blends has a neighbour who drinks water. 

It has been asserted that Albert Einstein wrote this riddle early
during the 19th century. He said that 98% of the world population
would not be able to solve it. 

Erwin Kalvalagen pointed me to a web site which comments on Einstein
being the author of this riddle (supposedly early on in his career):

"Finally, you may want to notice that the young Albert Einstein
(1879-1955) could not possibly have authored the puzzle in this form:
The Pall Mall brand of cigarettes was introduced by Butler & Butler in
1899 (sold to American Tobacco in 1907 and Brown & Williamson in 1994)
and Alfred Dunhill was established in 1893 (starting to manufacture
pipes in 1907) when Einstein was still a young man. However, the Blue
Master brand was introduced by J. L.  Tiedemann in 1937, when Einstein
was 58!"

Define five sets defining the five characteristics of
each individual.  The way the program is designed, the symbols
used to define characteristics must be distinct (e.g., you
cannot define an element of the smokes set S named "Red").

In [2]:
GU = GamsUniverse()

@GamsSet(GU,:h,"House",begin
    h1,""
    h2,""
    h3,""
    h4,""
    h5,""
end)

@GamsSet(GU,:c,"House Colors",begin
    red,""
    green,""
    yellow,""
    blue,""
    white,""
end)

@GamsSet(GU,:s,"Smokes",begin
    pall_mall,""
    dunhill,""
    blends,""
    prince,""
    blue_master,""
end)

@GamsSet(GU,:b,	"beverages",begin
    Coffee, ""
    Milk, ""
    Beer, ""
    Water, ""
    Tea, ""
end)

@GamsSet(GU,:p,"Pet",begin
    Dogs, ""
    Birds, ""
    Cats, ""
    Horses, ""
    Fish, ""
end)

@GamsSet(GU,:n, "Nationality",begin
    Brit,"" 
    Swede, ""
    Dane, ""
    Norwegian, ""
    German,""
end);

I'm using the GLPK solver since it supports binary variables, which this problem is all about. 

The idea here is to list every possible combination with a binary variable. Then use the constraints to fix many of these to be 0. The objective function is irrelevant, we are solely using the constraints to find where each entry is true.

In [35]:
function Riddle(GU)

    H = [h for h∈GU[:h]]
    C = [c for c∈GU[:c]]
    S = [s for s∈GU[:s]]
    B = [b for b∈GU[:b]]
    P = [p for p∈GU[:p]]
    N = [n for n∈GU[:n]]

    r = Model(GLPK.Optimizer)

    @variable(r,Z[H,C,S,B,P,N],Bin) #Choice variable is 1 if we have h-c-s-b-p-n living on street

    @objective(r,Min,0)

    @constraints(r,begin
        housing[h=H],   sum(Z[h,C,S,B,P,N])==1
        colors[c=C],    sum(Z[H,c,S,B,P,N])==1
        smokes[s=S],    sum(Z[H,C,s,B,P,N])==1
        beverages[b=B], sum(Z[H,C,S,b,P,N])==1
        pets[p=P],      sum(Z[H,C,S,B,p,N])==1
        nations[n=N],   sum(Z[H,C,S,B,P,n])==1
    end)

    #4. The Green house is just to the left of the White house.
    right_house = Dict(zip(H[1:end-1],H[2:end]))
    left_house = Dict(zip(H[2:end],H[1:end-1]))

    @constraint(r, hint4[h=H[1:end-1]],
        sum(Z[h,[:green],S,B,P,N]) <= sum(Z[right_house[h],[:white],S,B,P,N])
    )

    #10. The man who smokes Blends lives next to the one who keeps cats.
    @constraint(r, hint10[h=H[2:end-1]],
        sum(Z[h,C,[:blends],B,P,N]) <=
        sum(Z[right_house[h],C,S,B,[:Cats],N]) +sum(Z[left_house[h],C,S,B,[:Cats],N]) 
    )

    #11. The man who keeps horses lives next to the man who smokes Dunhill.
    @constraint(r, hint11[h=H[2:end-1]],
        sum(Z[h,C,S,B,[:Horses],N]) <=
        sum(Z[right_house[h],C,[:dunhill],B,P,N]) +sum(Z[left_house[h],C,[:dunhill],B,P,N]) 
    )  

    #14. The Norwegian lives next to the blue house.
    @constraint(r, hint14[h=H[2:end-1]],
        sum(Z[h,C,S,B,P,[:Norwegian]]) <=
        sum(Z[right_house[h],[:blue],S,B,P,N]) +sum(Z[left_house[h],[:blue],S,B,P,N]) 
    )     
    

    #15. The man who smokes Blends has a neighbour who drinks water. 
    @constraint(r, hint15[h=H[2:end-1]],
        sum(Z[h,C,[:blends],B,P,N]) <=
        sum(Z[right_house[h],C,S,[:Water],P,N]) +sum(Z[left_house[h],C,S,[:Water],P,N]) 
    )    

    for h∈H,c∈C,s∈S,b∈B,p∈P,n∈N
        #hint 1: The Brit lives in a red house. 
        if (c==:red) ⊻ (n == :Brit)
            fix(Z[h,c,s,b,p,n],0,force=true)
        end

        #2. The Swede keeps Dogs as pets. 
        if (n == :Swede) ⊻ (p==:Dogs)
            fix(Z[h,c,s,b,p,n],0,force=true)
        end

        #3. The Dane drinks tea.
        if (n==:Dane) ⊻ (b ==:Tea)
            fix(Z[h,c,s,b,p,n],0,force=true)
        end

        #5. The owner of the Green house drinks coffee. 
        if (c==:green) ⊻ (b==:Coffee)
            fix(Z[h,c,s,b,p,n],0,force=true)
        end

        #6. The person who smokes Pall Mall rears Birds. 
        if (s == :pall_mall) ⊻ (p==:Birds)
            fix(Z[h,c,s,b,p,n],0,force=true)
        end

        #7. The owner of the Yellow house smokes Dunhill. 
        if (c == :yellow) ⊻ (s==:dunhill)
            fix(Z[h,c,s,b,p,n],0,force=true)
        end

        #8. The man living in the centre house drinks milk. 
        if (h==:h3) ⊻ (b==:Milk)
            fix(Z[h,c,s,b,p,n],0,force=true)
        end

        #9. The Norwegian lives in the first house. 
        if (h==:h1) ⊻ (n==:Norwegian)
            fix(Z[h,c,s,b,p,n],0,force=true)
        end

        #12. The man who smokes Blue Master drinks beer. 
        if (s==:blue_master) ⊻ (b==:Beer)
            fix(Z[h,c,s,b,p,n],0,force=true)
        end

        #13. The German smokes Prince. 
        if (n==:German) ⊻ (s==:prince)
            fix(Z[h,c,s,b,p,n],0,force=true)
        end
    end




    return r
end

Riddle (generic function with 1 method)

In [39]:
riddle = Riddle(GU)
set_silent(riddle)
optimize!(riddle)


In [40]:
X = value.(riddle[:Z])

for h∈GU[:h],c∈GU[:c],s∈GU[:s],b∈GU[:b],p∈GU[:p],n∈GU[:n]
    if X[h,c,s,b,p,n] !=0
        println("$h, $c, $s, $b, $p, $n")
    end
end

h1, blue, blends, Water, Horses, Norwegian
h2, yellow, dunhill, Tea, Cats, Dane
h3, red, pall_mall, Milk, Birds, Brit
h4, green, prince, Coffee, Fish, German
h5, white, blue_master, Beer, Dogs, Swede
